In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType

In [ ]:
import numpy as np

In [ ]:
width = 0.1
thickness = width
cross_section = [thickness, width]

In [ ]:
input_path = '../../data/one.json'
from load_jsondata import read_data
input_data = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(input_data['vertices'], input_data['edges'], 5, input_data['alphas'], input_data['ghost_bisectors'], input_data['ghost_normals'], input_data['A_segments'], input_data['B_segments'], input_data['midpoint_offsets_A'], input_data['midpoint_offsets_B'], input_data['segment_normals'], input_data['is_rigid'], input_data['uid'], input_data['uid_top_bot_map'], input_data['color'])

In [ ]:
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
# view.setCameraParams(camParam)
view.show()

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1

#### Perturbation

In [ ]:
dof = curr_um.getDoFs()

In [ ]:
alphas = []
for i in range(curr_um.numJoints()):
    alphas.append(dof[curr_um.dofOffsetForJoint(i) + 6])

In [ ]:
for i in range(curr_um.numJoints()):
    dof[curr_um.dofOffsetForJoint(i) + 6] += 1

In [ ]:
curr_um.setDoFs(dof)

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
view.update(scalarField = rod_colors)

In [ ]:
# view.update(scalarField=curr_um.maxBendingStresses())

### Equilibrium solve

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 300
OPTS.verboseNonPosDef = False

#### Alpha deployment

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import importlib, open_umbrella
importlib.reload(open_umbrella)
from open_umbrella import open_umbrella

In [ ]:
from time import sleep
# sleep(0.5)
def equilibriumSolver(tgtAngle, um, opts, fv):
    opts.gradTol = 1e-4
    return umbrella_mesh.compute_equilibrium(um, tgtAngle, options=opts, fixedVars=fv)
driver = curr_um.centralJoint()
open_umbrella(curr_um, driver, np.deg2rad(180) - curr_um.averageJointAngle, 20, view, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, useTargetAngleConstraint=True, fixedVars = fixedVars)

#### Distance Deployment

In [ ]:
def eqm_callback(prob, i):
    view.update(scalarField = rod_colors)

In [ ]:
from time import sleep
sleep(2)
umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars)